In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.metrics import pairwise_distances

import warnings
warnings.filterwarnings("ignore")

from google.colab import files
util = files.upload()
transformed_model_data = files.upload()
transformed_test_data = files.upload()

Saving util.py to util.py


Saving transformed_model_data_t1.csv to transformed_model_data_t1.csv


Saving transformed_test_data_t1.csv to transformed_test_data_t1.csv


In [ ]:
from util import *

In [ ]:
df = pd.read_csv("transformed_model_data_t1.csv")
df_test = pd.read_csv("transformed_test_data_t1.csv")

# Popularity

In [ ]:
exp_df = pd.DataFrame(columns=["model", "hit", "map", "ndcg"])
cust_base = df_test["c_id"].unique()


k = 20

hitt = 0
pret = 0
ndct = 0

ret = df["item_id"].value_counts().keys()[:k]
counter = len(cust_base)
for i in range(counter):
    c = cust_base[i]
    all = df_test.loc[df_test["c_id"]==c, "item_id"].values
    rel = all[-1]
    bl = np.isin(ret, rel)*1
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    hitt += h
    pret += pre
    ndct += ndc
hit, mapp, ndcg = hitt/counter, pret/counter, ndct/counter
exp_df = exp_df.append({"model": "popularity",
                        "hit": hit,
                        "map": mapp,
                        "ndcg": ndcg}, ignore_index=True)


file = "".join(np.random.choice(["a", "b", "c", "d", "e", 0, 1, 2, 3, 4], 4))
exp_df.to_csv("results_" + file + ".csv", sep=";", index=False)

# Last-random

In [ ]:
exp_df = pd.DataFrame(columns=["model", "hit", "map", "ndcg"])
cust_base = df_test["c_id"].unique()

k = 20

hitt = 0
pret = 0
ndct = 0

counter = len(cust_base)
for i in range(counter):
    c = cust_base[i]
    all = df_test.loc[df_test["c_id"]==c, "item_id"].values
    rel = all[-1]
    rest = all[:-1]
    re = list()
    [re.append(r) if not r in re else False for r in rest]
    if len(re) > k:
        ret = np.random.choice(re, k, replace=False)
    else:
        ret = list()
        while len(re)>0:
            ri = re.pop()
            if ri not in ret:
                ret.append(ri)
        bl = np.isin(ret, rel)*1
        pre = average_precision(bl)
        ndc = ndcg_k(bl, len(bl))
        h = np.max(bl)
        hitt += h
        pret += pre
        ndct += ndc
hit, mapp, ndcg = hitt/counter, pret/counter, ndct/counter
exp_df = exp_df.append({"model": "last-random",
                        "hit": hit,
                        "map": mapp,
                        "ndcg": ndcg}, ignore_index=True)

file = "".join(np.random.choice(["a", "b", "c", "d", "e", 0, 1, 2, 3, 4], 4))
exp_df.to_csv("results_" + file + ".csv", sep=";", index=False)

# Last-ordered

In [ ]:
exp_df = pd.DataFrame(columns=["model", "hit", "map", "ndcg"])
cust_base = df_test["c_id"].unique()

k = 20

hitt = 0
pret = 0
ndct = 0

counter = len(cust_base)
for i in range(counter):
    c = cust_base[i]
    all = df_test.loc[df_test["c_id"]==c, "item_id"].values
    rel = all[-1]
    rest = all[:-1]
    re = list()
    [re.append(r) if not r in re else False for r in rest]
    ret = re[-k:]
    bl = np.isin(ret, rel)*1
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    hitt += h
    pret += pre
    ndct += ndc
hit, mapp, ndcg = hitt/counter, pret/counter, ndct/counter
exp_df = exp_df.append({"model": "last-ordered",
                        "hit": hit,
                        "map": mapp,
                        "ndcg": ndcg}, ignore_index=True)

file = "".join(np.random.choice(["a", "b", "c", "d", "e", 0, 1, 2, 3, 4], 4))
exp_df.to_csv("results_" + file + ".csv", sep=";", index=False)

In [ ]:
exp_df = pd.DataFrame(columns=["model", "hit", "map", "ndcg"])
cust_base = df_test["c_id"].unique()
model_data = df.append(df_test)

lasts = df_test.groupby("c_id", as_index=False).last()
last_tr = lasts["t_id"].unique()
train_data = model_data[~(model_data["t_id"].isin(last_tr))]


ns = [100]
k = 20

user = pd.pivot_table(data=train_data,
                      values="payment",
                      index="c_id",
                      columns="item_id")
user = user.apply(lambda x: x/x, axis=1)
user = user.fillna(0)
for n in ns:
    hitt = 0
    pret = 0
    ndct = 0 
    counter = len(cust_base)
    for i in range(counter):
        if (i%1000)==0:
            print(i)
        c = cust_base[i]
        all = df_test.loc[df_test["c_id"]==c, "item_id"].values
        rel = all[-1]
        score = 1 - pairwise_distances(user[user.index==c], user, metric="cosine")
        nn = user.index[score[0].argsort()[::-1][1:n+1]]
        rank_table = list()
        for h in nn: 
            rank_table.append(user[user.index==h].values)
        ranks = np.mean(rank_table, axis=0)[0].argsort()[::-1][:k]
        ret = list(user.columns[ranks])
        bl = np.isin(ret, rel)*1
        pre = average_precision(bl)
        ndc = ndcg_k(bl, len(bl))
        h = np.max(bl)
        hitt += h
        pret += pre
        ndct += ndc
    hit, mapp, ndcg = hitt/counter, pret/counter, ndct/counter
    exp_df = exp_df.append({"model": "user-similarity",
                            "n": n,
                            "hit": hit,
                            "map": mapp,
                            "ndcg": ndcg}, ignore_index=True)

file = "".join(np.random.choice(["a", "b", "c", "d", "e", 0, 1, 2, 3, 4], 4))
exp_df.to_csv("results_" + file + ".csv", sep=";", index=False)

0
1000
2000
